In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
from selenium.webdriver.common.by import By

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

import warnings
warnings.filterwarnings('ignore')
import time

The "requests" module allows you to send HTTP requests and handle the response. With this module, you can make GET, POST, PUT, DELETE requests to a web server.

The "urllib.request" module is part of the Python standard library and provides a way to open URLs (Uniform Resource Locators) and interact with the web.

The "os" module provides a way to interact with the operating system. You can use it to create, remove, and manipulate files and directories.

The "PIL" module (Python Imaging Library) is an image processing library that allows you to open, manipulate, and save image files in various formats.

The "io" module provides a way to read and write binary data. It is used to handle binary streams of data, such as image files.

The "re" module (regular expressions) is a powerful tool for text processing. You can use regular expressions to find, match, and replace patterns in strings.

With these modules you can scrape data from websites, process and save images and manipulate text.

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://www.amazon.in/"
driver.get(url)
time.sleep(2)

user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : guitar


In [3]:
search_bar = driver.find_element(By.ID,"twotabsearchtextbox")   
search_bar.clear()                                               
search_bar.send_keys(user_inp)                                   
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')     
search_button.click()   

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [4]:
# scrape all product urls
product_urls = []
start=0
end=3
for page in range(start,end):                                                 
    url=driver.find_elements(By.XPATH,'//a[@class = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href"))                        
    nxt_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')     #scraping the list of buttons from the page
    nxt_button.click()
    time.sleep(2)

In [5]:
len(product_urls)

204

In [6]:
product = []
price = []
availability=[]
Brand=[]
ReturnExchange=[]
Delivery=[]
Ratings=[]


for url in product_urls[:10]:                                                        
    driver.get(url)
    time.sleep(2)
    
    try:
        brand= driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-') 
        
    try:
        exchange = driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        ReturnExchange.append(exchange.text)
    except NoSuchElementException:
        ReturnExchange.append('-') 
        
    try:
        delivery= driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
        Delivery.append(delivery.text)
    except NoSuchElementException:
        Delivery.append('-') 
        
    try:
        product.append(driver.find_element(By.XPATH,'//span[@id="productTitle"]').text)
    except:
        product.append('-')
      
    try:
        try:
            price.append(driver.find_element(By.XPATH,'//td[@class="a-span12"]').text)
        except:
            price.append(driver.find_element(By.CSS_SELECTOR,'span[class="a-price-whole"]').text)             
    except:
        price.append('-')
    
    try:
        availability.append(driver.find_element(By.XPATH,'//div[@id="availability"]').text)
    except:
        availability.append('-')
        
    try:
        Ratings.append(driver.find_element(By.XPATH,'//span[@class="a-size-medium"]').text)
    except:
        Ratings.append('-')
         

In [7]:
print(len(Brand),len(ReturnExchange),len(Delivery),len(product),len(price),len(availability),len(Ratings))

10 10 10 10 10 10 10


In [8]:
df=pd.DataFrame({'Brand':Brand,'ReturnExchange':ReturnExchange,'Price':price,'Delivery_by':Delivery,'urls':url,'Ratings':Ratings,'Name of Product':product,'Availability of product':availability})
df

,Brand,ReturnExchange,Price,Delivery_by,urls,Ratings,Name of Product,Availability of product
0,Kadence,7 days Replacement,"9,999","Thursday, 2 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Kadence Slowhand Premium Jumbo Semi Acoustic G...,In stock.
1,Multicolor,7 days Replacement,449,"Wednesday, 1 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"RK Shop String Acoustic Guitar , Learning Toys...",In stock.
2,VAULT,7 days Replacement,"4,240","Wednesday, 1 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Vault DA20 Dreadnought Acoustic Guitar With Gi...,In stock.
3,Brown,7 days Returnable,740,"Saturday, 4 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"CHARCOALE String Acoustic Guitar, Learning Toy...",In stock.
4,JUAREZ,7 days Replacement,"2,148","Wednesday, 1 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",In stock.
5,Intern,7 days Replacement,"2,248","Wednesday, 1 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"Intern INT-38C Right hand Acoustic Guitar Kit,...",In stock.
6,YAMAHA,7 days Replacement,"7,490","Friday, 3 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,"Yamaha F280 Acoustic Rosewood Guitar (Natural,...",In stock.
7,Kadence,7 days Replacement,"9,999","Thursday, 2 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Kadence Slowhand Premium Jumbo Semi Acoustic G...,In stock.
8,Cort,7 days Replacement,"10,890","Wednesday, 1 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Cort AD810 Dreadnought Acoustic Guitar with Gi...,In stock.
9,KETOSTICS,7 days Replacement,"3,999","Friday, 3 February",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,-,Ketostics Givson Venus Special Guitar Combo Ac...,Only 2 left in stock.


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [21]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("fruits")

In [23]:
search_button = driver.find_element(By.XPATH,'//div[@class="zgAlFc"]')
search_button.click()

In [24]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Dimple Makwana\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [25]:
driver.close()

In [26]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("cars") 

In [28]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [30]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Dimple Makwana\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [31]:
driver.close()

In [32]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("Machine Learning") 

In [34]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [35]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Dimple Makwana\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

In [36]:
driver.close()

In [37]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://images.google.com/"
driver.get(url)
search_bar = driver.find_element(By.XPATH,'//input[@class="gLFyf"]')    
search_bar.send_keys("Guitar") 

In [38]:
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')   
search_button.click()

In [39]:
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
img = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in img:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\Dimple Makwana\Downloads"+str(i)+".jpg", "wb")
    file.write(response.content)

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be  scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CS

In [41]:
driver4=webdriver.Chrome('chromedriver.exe')
driver4.get("https://www.flipkart.com/")
time.sleep(2)
driver4.maximize_window()
pop_close = driver4.find_element("xpath","/html/body/div[2]/div/div/button").click()
product = input(" Enter the name of Smartphone that has to be searched : ")
search_product4=driver4.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone that has to be searched : mi 11x


In [42]:
search_button4= driver4.find_element(By.CLASS_NAME,'L0Z3Pu').click()

In [43]:
page1_urls = []
urls = driver4.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [44]:
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [45]:
for url in page1_urls:
    driver4.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver4.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver4.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #Scraping Display size data 
    try:
        disp_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #Scraping battery capacity data
    try:
        if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #Scraping Price data
    try:
        price_tags = driver4.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [46]:
smartphone=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

smartphone

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,Mi,11X,Lunar White,6 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹24,449",https://www.flipkart.com/mi-11x-lunar-white-12...
1,Mi,11X,Cosmic Black,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹24,990",https://www.flipkart.com/mi-11x-cosmic-black-1...
2,Mi,11X,Cosmic Black,8 GB,128 GB,Primary Camera,-,-,-,"₹24,699",https://www.flipkart.com/mi-11x-cosmic-black-1...
3,Mi,11X,Cosmic Black,6 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹28,499",https://www.flipkart.com/mi-11x-cosmic-black-1...
4,Mi,11X,Cosmic Black,8 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹26,499",https://www.flipkart.com/mi-11x-cosmic-black-1...
5,Mi,11X,Celestial Silver,6 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹24,490",https://www.flipkart.com/mi-11x-celestial-silv...
6,Mi,11X,Lunar White,8 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹23,990",https://www.flipkart.com/mi-11x-lunar-white-12...
7,Mi,11X,Celestial Silver,8 GB,128 GB,48MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹26,149",https://www.flipkart.com/mi-11x-celestial-silv...
8,Mi,11X Pro 5G,Cosmic Black,8 GB,128 GB,108MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹30,990",https://www.flipkart.com/mi-11x-pro-5g-cosmic-...
9,Mi,11X Pro 5G,Lunar White,8 GB,128 GB,108MP + 8MP + 5MP,20MP Front Camera,-,4520 mAh,"₹31,990",https://www.flipkart.com/mi-11x-pro-5g-lunar-w...


In [47]:
smart_phone=smartphone.to_csv('smart_phone.csv')
smart_phone

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.


In [48]:
driver5=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps web page
driver5.get("https://www.google.co.in/maps")
driver5.maximize_window()
time.sleep(2)

#Sending keyword for seach box and search button
city = input('Enter City name to search : ')
search_city = driver5.find_element(By.ID,"searchboxinput")                       
search_city.clear()                                                             
time.sleep(2)

search_city.send_keys(city)                                                     
search_button = driver5.find_element(By.ID,"searchbox-searchbutton").click()                                                             
time.sleep(3)
Location=driver5.current_url

try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={}".format(lat))
print("\nlongitude={}".format(lon))

Enter City name to search : Vadodara

latitude=22.3220876

longitude=73.1030456


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [62]:
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of googlemaps
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [67]:
button = driver.find_element(By.XPATH,'//li[@id="menu-item-1237902"]/a').get_attribute('href')
driver.get(button)

In [69]:
date=[]
name=[]
industry=[]
subvertical=[]
location=[]
investors_name=[]
investors_type=[]
amount=[]

In [70]:
dates=driver.find_elements(By.XPATH,'//td[@class="column-2"]')
for i in dates[5:29]:
       date.append(i.text)
names=driver.find_elements(By.XPATH,'//td[@class="column-3"]')
for i in names[5:29]:
       name.append(i.text)
industrys=driver.find_elements(By.XPATH,'//td[@class="column-4"]')
for i in industrys[5:29]:
       industry.append(i.text)
subverticals=driver.find_elements(By.XPATH,'//td[@class="column-5"]')
for i in subverticals[5:29]:
       subvertical.append(i.text)
locations=driver.find_elements(By.XPATH,'//td[@class="column-6"]')
for i in locations[5:29]:
       location.append(i.text)
investors_names=driver.find_elements(By.XPATH,'//td[@class="column-7"]')
for i in investors_names[5:29]:
       investors_name.append(i.text)
investors_types=driver.find_elements(By.XPATH,'//td[@class="column-8"]')
for i in investors_types[5:29]:
       investors_type.append(i.text)
amounts=driver.find_elements(By.XPATH,'//td[@class="column-9"]')
for i in amounts[5:29]:
       amount.append(i.text)

In [71]:
print(len(date),len(name),len(industry),len(subvertical),len(location),len(investors_name),len(investors_type),len(amount))


0 0 0 0 0 0 0 0


In [72]:
df=pd.DataFrame({'Date':date,'Name':name,'Industry':industry,'Subvertical':subvertical,'Location':location,'Investors_name':investors_name,'Investors_type':investors_type,'Amount':amount})
df

,Date,Name,Industry,Subvertical,Location,Investors_name,Investors_type,Amount


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [79]:
# let's first connect to the web driver
driver = webdriver.Chrome(r"C:\Users\Sunny\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

# Opening the homepage of digit.in
url = "https://digit.in"
driver.get(url)
time.sleep(2)

In [80]:
top_10=driver.find_element(By.XPATH,'//div[@class="menu"]//ul//li[4]')
top_10.click()

In [83]:
laptops=driver.find_element(By.XPATH,"/html/body/div[4]/div/div/div[3]/div[2]/ul/li[22]/a")
laptops.click()

In [85]:
name = []
Price = []
OS = []
display = []
processor = []
weight = []
dimension = []

In [86]:
names=driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)

time.sleep(1)
#scraping operating system
os=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)

time.sleep(1)
#scraping display
displays=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)

time.sleep(1)
#scraping processor
processors=driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)


time.sleep(1)
# scraping weight
weights=driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")# extrat weight form xpath
for i in weights:
    weight.append(i.text)

time.sleep(1)
# scraping dimension
dimension=[]
dimensions=driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]") 
for i in dimensions:
    dimension.append(i.text)

time.sleep(1)

#scraping price
price=driver.find_elements(By.XPATH,"//table[@id='summtable']//tr//td[3]")
for i in price[]:
    Price.append(i.text)

In [87]:
print(len(name),len(Price),len(OS),len(display),len(weight),len(dimension),len(processor))


0 10 10 10 10 9 0


In [89]:
df=pd.DataFrame({'Price':Price,'OS':OS,'display':display,'weight':weight,})
df

,Price,OS,display,weight
0,N/A,Windows 11 Home OS,"17.3"" (3840 x 2160) Display",397 x 330 x 23 mm dimension & 3.3 kg weight
1,"₹ 389,990",Windows 11 Home OS,"17.3"" (1920 x 1080) Display",299 x 359 x 21 mm dimension & 2.96 kg weight
2,"₹ 229,990",Windows 11 OS,"16"" (2560 x 1600) Display",262 x 359 x 20 mm dimension & 2.4 kg weight
3,"₹ 176,490",Windows 11 Home OS,"16.1"" (2560 x 1440) Display",369 x 248 x 23 mm dimension & 2.32 kg weight
4,"₹ 172,999",Windows 11 Home OS,"15.6"" (2560 x 1440) Display",363 x 255 x 23.9 mm dimension & 2.4 kg weight
5,"₹ 188,990",Windows 11 Home OS,"15.6"" (1920 x 1080) Display",357 x 247 x 19 mm dimension & 1.9 kg weight
6,"₹ 124,990",Windows 11 Home OS,"16.1"" (2560 x 1440) Display",36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight
7,"₹ 167,990",Windows 11 Home OS,"16"" (2560 x 1600) Display",360 x 264 x 20 mm dimension & 2.49 kg weight
8,"₹ 174,000",Windows 11 Home OS,"15.6"" (1920 x 1080) Display",356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...
9,"₹ 131,990",Windows 11 Home OS,15.6 MP | NA Display,356 x 252 x 16 mm dimension & 1.9 kg weight


In [90]:
driver.close()

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [91]:
driver8=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening web page
driver8.get("https://www.forbes.com/billionaires/")
driver8.maximize_window()
time.sleep(2)

In [92]:
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []

#Fetching Rank
rank_tag = driver8.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")
    
#Fetching Name
name_tag = driver8.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
#Fetching Net Worth
netwrth_tag = driver8.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")
        
#Fetching Age
age_tag = driver8.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")
    
#Fetching Citizenship
cit_tag = driver8.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")
    
#Fetching Source
src_tag = driver8.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")
    
#Fetching Industry
ind_tag = driver8.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")
        


In [93]:
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)

(200, 200, 200, 200, 200, 200, 200)

In [94]:
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [96]:
driver9=webdriver.Chrome('chromedriver.exe')
driver9.get("https://www.youtube.com/watch?v=AETFvQonfV8")
driver9.maximize_window()
time.sleep(2)

In [105]:
for _ in range(500):
    driver9.execute_script("window.scrollBy(0,500)")

In [108]:
i=0
while(i<100):
    driver9.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver9.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
    
comment = []
upvote = []
comment_time = []

comment_tag=(driver9.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag[0:501]:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")

upvote_tag=(driver9.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag[0:501]:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")

comment_time_tag=(driver9.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag[0:501]:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")

In [109]:
len(comment),len(upvote),len(comment_time)

(501, 501, 501)

In [110]:
video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video

,Comment,Upvote,Comment_ Time
0,Hanuman chalisa kisko kisko pasand lga \n\n\nL...,535,3 years ago
1,जब जब भी मैं हनुमान चालीसा को सुनता हूं मेरे म...,213,5 days ago (edited)
2,Jai hanuman ️,6,1 day ago
3,श्री हनुमान चालीसा ️\nदोहा️\nश्रीगुरु चरन सरोज...,93,10 hours ago
4,Jai Siyaram ️ Jai Shree Samodh Vale Balaji Ki ...,7,17 hours ago
...,...,...,...
496,जब भी सुनता हूं मन को शांति मिलती है\nजय श्री ...,14,5 days ago
497,Nice beautiful,,7 hours ago
498,Friends listen everyday in early morning it g...,32,1 day ago
499,Jai shree ram \nJai shree balaji,,2 hours ago


# Q.10. Answer: Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [112]:
driver=webdriver.Chrome('chromedriver.exe')
time.sleep(3)

url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(2)

In [115]:
london_tag=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input')
london_tag.send_keys('London')


In [117]:
london_btn=driver.find_element(By.XPATH,"li[2]/div")
london_btn.click() 


In [118]:
go_btn=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[5]/button")
go_btn.click() 

In [119]:
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [126]:
while(True):
    
    names = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    dis = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    review = driver.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    t_review = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    service = driver.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    prices = driver.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)        
        
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])
time.sleep(2)

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])

In [127]:
hostel_url = []

while(True):
    urls = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break

In [128]:
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    try:
        pd = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  
        
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [129]:
print(len(Hostel_Name),len(Distance),len(total_reviews),len(facilities),len(Privates_From),len(Dorms_From))

76 76 76 76 76 76


In [130]:
import pandas as pd
df=pd.DataFrame({'Name':Hostel_Name,'Distance_city':Distance,'Tot_review':total_reviews,'Pri_frm_price':Privates_From,'Dorms_frm_price':Dorms_From,'Facility':facilities})
df

,Name,Distance_city,Tot_review,Pri_frm_price,Dorms_frm_price,Facility
0,NX London Hostel,Hostel - 6.1km from city centre,1550 Total Reviews,Privates From\nRs6334.96 Rs4434,Dorms From\nRs1896.27 Rs1612,Free WiFi\nFree Breakfast
1,Wombat's City Hostel London,Hostel - 3.6km from city centre,14521 Total Reviews,Privates From\nRs12970.92 Rs10377,Dorms From\nRs3311.12 Rs2649,Free WiFi\nFollows Covid-19 sanitation guidance
2,St Christopher's Village,Hostel - 1.8km from city centre,11869 Total Reviews,Privates From\nRs6877.12 Rs6189,Dorms From\nRs1426.9 Rs1284,Free WiFi\nFollows Covid-19 sanitation guidance
3,Astor Hyde Park,Hostel - 4.3km from city centre,11870 Total Reviews,No Privates Available,Dorms From\nRs1785,Free WiFi\nFollows Covid-19 sanitation guidance
4,Hostel One Notting Hill,Hostel - 5.5km from city centre,1718 Total Reviews,Privates From\nRs8225,Dorms From\nRs2851,Free WiFi
...,...,...,...,...,...,...
71,Fairway Hotel,Bed and Breakfast - 3.3km from city centre,44 Total Reviews,Privates From\nRs9899,No Dorms Available,Free WiFi
72,Park Hotel,Hotel - 4.9km from city centre,0 Total Reviews,Privates From\nRs10909,No Dorms Available,Free WiFi
73,Enrico Hotel,Hotel - 2km from city centre,166 Total Reviews,Privates From\nRs9091,No Dorms Available,Free WiFi\nFree Breakfast
74,Budget London Apartments,Apartment - 8.2km from city centre,357 Total Reviews,Privates From\nRs5556,Dorms From\nRs1717,Free WiFi
